In [ ]:
# 🎯 Prompting Techniques Demo

**Learn how different prompting approaches affect AI responses!**

This notebook demonstrates 4 key prompting techniques using the same customer support email to show how the **way you ask** dramatically changes **what you get** from AI models.

## 📧 Customer Support Example

We'll use this customer email throughout all demonstrations:

> **Subject:** Order #12345 - Wrong Item Received
>
> Hi there,
>
> I ordered a blue sweater (size M) last week but received a red jacket (size L) instead. This is really frustrating because I needed it for an event tomorrow.
>
> Can you please help me get the right item or process a refund quickly?
>
> Best regards,  
> Sarah Johnson

**Goal:** Classify this email into categories: **Complaint**, **Question**, **Refund Request**, or **Compliment**


In [1]:
# Setup and Configuration
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️  Warning: OPENAI_API_KEY not found in environment variables")
    print("Please set your OpenAI API key in a .env file or environment variable")
    print("Get your key from: https://platform.openai.com/api-keys")
else:
    print("✅ OpenAI API key loaded successfully!")

# Customer email for all demonstrations
CUSTOMER_EMAIL = """
Subject: Order #12345 - Wrong Item Received

Hi there,

I ordered a blue sweater (size M) last week but received a red jacket (size L) instead. 
This is really frustrating because I needed it for an event tomorrow. 

Can you please help me get the right item or process a refund quickly?

Best regards,
Sarah Johnson
"""

print("🎯 Setup complete! Ready for prompting technique demonstrations.")


✅ OpenAI API key loaded successfully!
🎯 Setup complete! Ready for prompting technique demonstrations.


In [ ]:
## 🎯 1. Zero-Shot Prompting

**Definition:** Ask the AI to perform a task without providing any examples.

**Characteristics:**
- ✅ **Simple and direct** - Straightforward request
- ✅ **Relies on pre-trained knowledge** - Uses what the model already knows
- ✅ **Quick and efficient** - Good for straightforward tasks
- ❌ **Less predictable** - Output format may vary

**When to use:** Simple, direct tasks when you trust the AI's built-in knowledge.


In [2]:
# 🎯 Zero-Shot Prompting Demo

print("🎯 ZERO-SHOT PROMPTING")
print("=" * 50)
print("Direct question with no examples or special instructions")
print()

# Create a simple, direct prompt
prompt = ChatPromptTemplate([
    ("user", "Classify this customer email into one of these categories: "
              "Complaint, Question, Refund Request, or Compliment.\n\n"
              "Email: {email}")
])

# Initialize model and create chain
model = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=OPENAI_API_KEY)
chain = prompt | model | StrOutputParser()

print("📧 Customer Email:")
print(CUSTOMER_EMAIL)
print("\n🤖 AI Response (Zero-shot):")

# Get response
result = chain.invoke({"email": CUSTOMER_EMAIL})
print(result)

print("\n💡 Notice: Simple, direct classification with minimal explanation")


🎯 ZERO-SHOT PROMPTING
Direct question with no examples or special instructions

📧 Customer Email:

Subject: Order #12345 - Wrong Item Received

Hi there,

I ordered a blue sweater (size M) last week but received a red jacket (size L) instead. 
This is really frustrating because I needed it for an event tomorrow. 

Can you please help me get the right item or process a refund quickly?

Best regards,
Sarah Johnson


🤖 AI Response (Zero-shot):
Category: Complaint

💡 Notice: Simple, direct classification with minimal explanation


In [ ]:
## 📚 2. Few-Shot Prompting

**Definition:** Provide examples before asking the AI to perform the task.

**Characteristics:**
- ✅ **Shows expected format** - Examples demonstrate desired output style
- ✅ **More consistent results** - AI follows the pattern from examples  
- ✅ **Better for specific formatting** - When you want particular response structure
- ❌ **Takes more tokens** - Examples use up context space

**When to use:** When you want consistent formatting or specific output style.


In [ ]:
# 📚 Few-Shot Prompting Demo

print("📚 FEW-SHOT PROMPTING")
print("=" * 50)
print("Provide examples to show the AI how to respond")
print()

# Create examples of email classifications
examples = [
    {
        "email": "Subject: Great Service!\nYour team was amazing. Fast delivery and perfect product!",
        "classification": "Compliment - Customer expressing satisfaction with service and product"
    },
    {
        "email": "Subject: Question about shipping\nHow long does international shipping usually take?",
        "classification": "Question - Customer asking for information about shipping times"
    },
    {
        "email": "Subject: Return my purchase\nI want to return this item and get my money back.",
        "classification": "Refund Request - Customer wants to return product and get refund"
    }
]

print("📚 Examples provided to AI:")
for i, example in enumerate(examples, 1):
    print(f"{i}. {example['email'][:50]}... → {example['classification']}")

# Create few-shot prompt template
example_prompt = ChatPromptTemplate([
    ("user", "Email: {email}"),
    ("assistant", "Classification: {classification}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate([
    few_shot_prompt,
    ("user", "Email: {email}")
])

# Create chain and get response
model = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=OPENAI_API_KEY)
chain = final_prompt | model | StrOutputParser()

print("\n📧 New Customer Email:")
print(CUSTOMER_EMAIL[:100] + "...")
print("\n🤖 AI Response (Few-shot):")

result = chain.invoke({"email": CUSTOMER_EMAIL})
print(result)

print("\n💡 Notice: More detailed classification following the example format")


In [ ]:
## 🧠 3. Chain-of-Thought (CoT) Prompting

**Definition:** Ask the AI to show its reasoning step by step.

**Characteristics:**
- ✅ **Transparent decision-making** - Shows how AI reaches conclusions
- ✅ **Better for complex problems** - Breaks down thinking process
- ✅ **Helps verify logic** - You can check the AI's reasoning
- ❌ **Longer responses** - More verbose output

**When to use:** Complex reasoning, problem-solving, or when you need to verify the AI's logic.


In [ ]:
# 🧠 Chain-of-Thought (CoT) Prompting Demo

print("🧠 CHAIN-OF-THOUGHT (CoT) PROMPTING")
print("=" * 50)
print("Ask AI to explain its reasoning step by step")
print()

# Prompt that asks for step-by-step reasoning
prompt = ChatPromptTemplate([
    ("user", "Classify this customer email into one of these categories: "
              "Complaint, Question, Refund Request, or Compliment.\n\n"
              "Think step by step:\n"
              "1. First, identify the main issue or request\n"
              "2. Look for emotional tone (frustrated, happy, neutral)\n"
              "3. Determine what action the customer wants\n"
              "4. Choose the most appropriate category\n"
              "5. Explain your reasoning\n\n"
              "Email: {email}")
])

model = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=OPENAI_API_KEY)
chain = prompt | model | StrOutputParser()

print("📧 Customer Email:")
print(CUSTOMER_EMAIL[:100] + "...")
print("\n🤖 AI Response (Chain-of-Thought):")

result = chain.invoke({"email": CUSTOMER_EMAIL})
print(result)

print("\n💡 Notice: Step-by-step reasoning process shown before final classification")


In [ ]:
## 👤 4. Role Prompting

**Definition:** Ask the AI to act as a specific expert or persona.

**Characteristics:**
- ✅ **Leverages domain expertise** - AI adopts specialist knowledge
- ✅ **More contextually appropriate** - Responses fit the professional role
- ✅ **Better understanding of nuances** - Professional perspective
- ❌ **May be overly formal** - Sometimes less conversational

**When to use:** Domain-specific expertise needed, when context and professional perspective matter.


In [ ]:
# 👤 Role Prompting Demo

print("👤 ROLE PROMPTING")
print("=" * 50)
print("Ask AI to act as a specific expert or persona")
print()

# Prompt that assigns a specific role to the AI
prompt = ChatPromptTemplate([
    ("system", "You are an experienced customer service manager with 10 years of experience "
               "handling customer emails. You understand customer emotions and business priorities. "
               "You're known for your ability to quickly assess situations and provide helpful guidance."),
    ("user", "As a customer service manager, classify this email and provide guidance on "
              "how urgent this case is and what action should be taken.\n\n"
              "Categories: Complaint, Question, Refund Request, or Compliment\n"
              "Urgency: Low, Medium, High\n"
              "Action: What should the support team do?\n\n"
              "Email: {email}")
])

model = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=OPENAI_API_KEY)
chain = prompt | model | StrOutputParser()

print("👤 AI Role: Experienced Customer Service Manager")
print("\n📧 Customer Email:")
print(CUSTOMER_EMAIL[:100] + "...")
print("\n🤖 AI Response (Role-based):")

result = chain.invoke({"email": CUSTOMER_EMAIL})
print(result)

print("\n💡 Notice: Professional assessment with urgency level and actionable recommendations")


In [ ]:
## 📊 Comparison Summary

Now that you've seen all four techniques in action, let's compare their strengths:

| Technique | Best For | Response Style | Complexity |
|-----------|----------|----------------|------------|
| **🎯 Zero-Shot** | Simple, direct tasks | Brief, factual | Low |
| **📚 Few-Shot** | Consistent formatting | Follows examples | Medium |
| **🧠 Chain-of-Thought** | Complex reasoning | Step-by-step logic | High |
| **👤 Role Prompting** | Professional context | Expert perspective | Medium |

## 💡 Key Takeaways

1. **Different techniques produce very different responses** to the same input
2. **Choose the right technique for your use case:**
   - Simple facts → Zero-shot
   - Specific format → Few-shot  
   - Complex analysis → Chain-of-thought
   - Professional context → Role prompting
3. **Prompt engineering is as important as the model itself**
4. **Experimentation is key** - try different approaches to find what works best

## 🚀 Next Steps

- Try modifying the customer email and see how responses change
- Experiment with different roles (teacher, lawyer, doctor)
- Combine techniques (e.g., role + chain-of-thought)
- Test with your own real-world use cases
